In [13]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate # few_shot 은 모델들에게 더 나은 대답을 하도록 예제들을 주는 개념 (프롬프트로 모든걸 제어하기보다 효율적)
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector # 제공할 예제를 형식화하기

# 제공하는 예제들중에 선택하기
class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

chat = ChatOpenAI(
    temperature=0.1,    # 숫자가 높을수록 랜덤함.
    streaming=True,     # 처리하는 과정을 살펴볼때 쓰는 옵션
    callbacks=[StreamingStdOutCallbackHandler()] # 처리하는 과정을 살펴볼때 쓰는 옵션2
    )    

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

# 제공되는 예제를 형식화하기
example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

# chain = prompt | chat

# chain.invoke({
#     #"country":"Germany"
#     #"country":"Korea"
#     "country":"Thailand"
# })

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'